In [1]:
# prerequisites
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image

bs = 100
# MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./mnist_data/', train=False, transform=transforms.ToTensor(), download=False)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=bs, shuffle=False)

100%|██████████| 9912422/9912422 [00:00<00:00, 119791254.87it/s]


Extracting ./mnist_data/MNIST/raw/train-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 97219657.96it/s]


Extracting ./mnist_data/MNIST/raw/train-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 29363974.63it/s]


Extracting ./mnist_data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 20911667.14it/s]


Extracting ./mnist_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw



In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class VAE(nn.Module):
    def __init__(self, x_dim, h_dim1, h_dim2, z_dim):
        super(VAE, self).__init__()

        # Parte del codificador (encoder)
        self.fc1 = nn.Linear(x_dim, h_dim1)
        self.fc2 = nn.Linear(h_dim1, h_dim2)
        self.fc31 = nn.Linear(h_dim2, z_dim)
        self.fc32 = nn.Linear(h_dim2, z_dim)

        # Parte del decodificador (decoder)
        self.fc4 = nn.Linear(z_dim, h_dim2)
        self.fc5 = nn.Linear(h_dim2, h_dim1)
        self.fc6 = nn.Linear(h_dim1, x_dim)

    def encoder(self, x):
        h = F.relu(self.fc1(x))
        h = F.relu(self.fc2(h))
        return self.fc31(h), self.fc32(h)  # mu, log_var

    def sampling(self, mu, log_var):
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu)  # devuelve una muestra z

    def decoder(self, z):
        h = F.relu(self.fc4(z))
        h = F.relu(self.fc5(h))
        return F.sigmoid(self.fc6(h))

    def forward(self, x):
        mu, log_var = self.encoder(x.view(-1, 784))
        z = self.sampling(mu, log_var)
        return self.decoder(z), mu, log_var

# Construir el modelo
vae = VAE(x_dim=784, h_dim1=512, h_dim2=256, z_dim=2)
if torch.cuda.is_available():
    vae.cuda()


In [5]:
vae

VAE(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc31): Linear(in_features=256, out_features=2, bias=True)
  (fc32): Linear(in_features=256, out_features=2, bias=True)
  (fc4): Linear(in_features=2, out_features=256, bias=True)
  (fc5): Linear(in_features=256, out_features=512, bias=True)
  (fc6): Linear(in_features=512, out_features=784, bias=True)
)

In [6]:
optimizer = optim.Adam(vae.parameters())

# Definición de la función de pérdida
def loss_function(recon_x, x, mu, log_var):
    # Término de error de reconstrucción usando binary cross entropy loss
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')

    # Término de divergencia de Kullback-Leibler
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())

    # La pérdida total es la suma de ambos términos
    return BCE + KLD


In [7]:
def train(epoch):
    # Poner el modelo en modo de entrenamiento
    vae.train()

    # Inicializar la pérdida de entrenamiento
    train_loss = 0

    # Iterar sobre lotes de datos de entrenamiento
    for batch_idx, (data, _) in enumerate(train_loader):
        # Mover los datos a la GPU si está disponible
        data = data.cuda()

        # Inicializar los gradientes en cero
        optimizer.zero_grad()

        # Obtener la reconstrucción, media y logaritmo de la varianza del VAE
        recon_batch, mu, log_var = vae(data)

        # Calcular la pérdida utilizando la función de pérdida definida
        loss = loss_function(recon_batch, data, mu, log_var)

        # Retropropagar el error y realizar una actualización de los parámetros
        loss.backward()
        train_loss += loss.item()
        optimizer.step()

        # Imprimir información de entrenamiento cada 100 lotes
        if batch_idx % 100 == 0:
            print('Época de Entrenamiento: {} [{}/{} ({:.0f}%)]\tPérdida: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item() / len(data)))

    # Imprimir la pérdida promedio al final de la época
    print('====> Época: {} Pérdida Promedio: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))


In [8]:
def test():
    # Poner el modelo en modo de evaluación
    vae.eval()

    # Inicializar la pérdida de prueba
    test_loss = 0

    # Desactivar el cálculo de gradientes durante la evaluación
    with torch.no_grad():
        # Iterar sobre lotes de datos de prueba
        for data, _ in test_loader:
            # Mover los datos a la GPU si está disponible
            data = data.cuda()

            # Obtener la reconstrucción, media y logaritmo de la varianza del VAE
            recon, mu, log_var = vae(data)

            # Sumar la pérdida del lote
            test_loss += loss_function(recon, data, mu, log_var).item()

    # Calcular la pérdida promedio de prueba
    test_loss /= len(test_loader.dataset)

    # Imprimir la pérdida del conjunto de prueba
    print('====> Pérdida del conjunto de prueba: {:.4f}'.format(test_loss))


In [9]:
for epoch in range(1, 51):
    train(epoch)
    test()

Época de Entrenamiento: 1 [0/60000 (0%)]	Pérdida: 547.177969
Época de Entrenamiento: 1 [10000/60000 (17%)]	Pérdida: 191.385195
Época de Entrenamiento: 1 [20000/60000 (33%)]	Pérdida: 172.237070
Época de Entrenamiento: 1 [30000/60000 (50%)]	Pérdida: 178.769414
Época de Entrenamiento: 1 [40000/60000 (67%)]	Pérdida: 163.700059
Época de Entrenamiento: 1 [50000/60000 (83%)]	Pérdida: 171.646211
====> Época: 1 Pérdida Promedio: 181.5036
====> Pérdida del conjunto de prueba: 163.3587
Época de Entrenamiento: 2 [0/60000 (0%)]	Pérdida: 157.692832
Época de Entrenamiento: 2 [10000/60000 (17%)]	Pérdida: 152.844844
Época de Entrenamiento: 2 [20000/60000 (33%)]	Pérdida: 158.344375
Época de Entrenamiento: 2 [30000/60000 (50%)]	Pérdida: 156.198555
Época de Entrenamiento: 2 [40000/60000 (67%)]	Pérdida: 162.212363
Época de Entrenamiento: 2 [50000/60000 (83%)]	Pérdida: 151.849609
====> Época: 2 Pérdida Promedio: 158.5638
====> Pérdida del conjunto de prueba: 155.1141
Época de Entrenamiento: 3 [0/60000 (0%)]

In [9]:
with torch.no_grad():
    z = torch.randn(64, 2).cuda()
    sample = vae.decoder(z).cuda()

    save_image(sample.view(64, 1, 28, 28), './sample_' + '.png')